In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
from ephysvibe.trials.spikes import firing_rate,sp_constants
# from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.task import def_task,task_constants
from collections import defaultdict
from typing import Dict
from ephysvibe.structures.trials_data import TrialsData
import os 

In [2]:
filepath ="/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/Riesling/lip/2022-12-01_12-01-59_Riesling_lip_e1_r1.h5"
s_path = os.path.normpath(filepath).split(os.sep)
data = TrialsData.from_python_hdf5(filepath)

In [28]:
b1 = np.where(data.block == 1)[0][0]
trial_idx = np.where(data.trial_error[b1:] == 0)[0] 

In [3]:
# Select trials and create task frame
trial_idx = np.where(np.logical_and(data.trial_error == 0, data.block == 1))[0]
if np.any(np.isnan(data.neuron_cond)):
    neuron_cond = np.ones(len(data.clustersgroup))
else:
    neuron_cond=data.neuron_cond
task = def_task.create_task_frame(
    condition=data.condition[trial_idx],
    test_stimuli=data.test_stimuli[trial_idx],
    samples_cond=task_constants.SAMPLES_COND,
    neuron_cond = neuron_cond,
)

cgroup = "good"
neurons = np.where(data.clustersgroup != cgroup)[0]
print("Number of clusters: %d" % len(data.clustersgroup))
print("Number of %s units: %d" % (cgroup, len(neurons)))

Number of clusters: 8
Number of good units: 2


In [5]:
if cgroup == 'all':
    neurons = np.where(data.clustersgroup != cgroup)[0]
else:
    neurons = np.where(data.clustersgroup == cgroup)[0]

In [11]:
in_out='in'
task=task[np.logical_and(np.in1d(task['i_neuron'].values,neurons[0]),task['in_out']==in_out)]

In [23]:
task_sample = task.loc[task['sample'].str.contains("c1", case=False)]
trials_n = task_sample['trial_idx'].values
#trials_n = trial_idx[trials_n]

In [33]:
data.test_stimuli[trial_idx[trials_n]]

array([[55., 31., 55., 11., nan],
       [11., nan, nan, nan, nan],
       [51., 61., 11., nan, nan],
       [11., nan, nan, nan, nan],
       [16., 62., 11., nan, nan],
       [11., nan, nan, nan, nan],
       [11., nan, nan, nan, nan],
       [41., 13., 86., 11., nan],
       [51., 11., nan, nan, nan],
       [13., 21., 11., nan, nan],
       [71., 11., nan, nan, nan],
       [12., 81., 11., nan, nan],
       [11., nan, nan, nan, nan],
       [33., 17., 51., 46., 51.],
       [11., nan, nan, nan, nan],
       [13., 71., 15., 11., nan],
       [31., 11., nan, nan, nan],
       [11., nan, nan, nan, nan],
       [61., 18., 11., nan, nan],
       [11., nan, nan, nan, nan],
       [11., nan, nan, nan, nan],
       [16., 11., nan, nan, nan],
       [11., nan, nan, nan, nan],
       [54., 81., 11., nan, nan],
       [11., nan, nan, nan, nan],
       [81., 55., 11., nan, nan],
       [11., nan, nan, nan, nan],
       [11., nan, nan, nan, nan],
       [51., nan, nan, nan, nan],
       [48., 5

In [34]:
task_sample

,i_neuron,trial_idx,sample,in_out,n_test_stimuli,code
31,1,2,o1_c1,in,4,4
32,1,5,o1_c1,in,1,1
34,1,14,o1_c1,in,3,3
35,1,22,o1_c1,in,1,1
36,1,24,o1_c1,in,3,3
37,1,30,o1_c1,in,1,1
38,1,44,o1_c1,in,1,1
39,1,49,o1_c1,in,4,4
40,1,63,o1_c1,in,2,2
41,1,66,o1_c1,in,3,3


In [18]:
data.code_numbers[trials_n]

array([[ 9.,  7., 35.,  8., 23., 24., 25., 26., 27., 28., 29., 30., 31.,
         4., 36., 32., 96., 18., nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25.,  4., 36., 26., 96., 18., nan,
        nan, nan, nan, nan, nan, nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25., 26., 27., 28., 29.,  4., 36.,
        30., 96., 18., nan, nan, nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25.,  4., 36., 26., 96., 18., nan,
        nan, nan, nan, nan, nan, nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25., 26., 27., 28., 29.,  4., 36.,
        30., 96., 18., nan, nan, nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25.,  4., 36., 26., 96., 18., nan,
        nan, nan, nan, nan, nan, nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25.,  4., 36., 26., 96., 18., nan,
        nan, nan, nan, nan, nan, nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25., 26., 27., 28., 29., 30., 31.,
         4., 36., 32., 96., 18., nan, nan],
       [ 9.,  7., 35.,  8., 23., 24., 25., 26., 27.,  4., 36., 2

In [15]:
trials_idx

array([  2,   5,  14,  22,  24,  30,  44,  49,  63,  66,  81,  83,  85,
       107, 127, 135, 143, 145, 158, 182, 184, 187, 196, 200, 222, 225,
       233, 235,   6,  10,  16,  39,  45,  52, 110, 115, 130, 138, 152,
       161, 180, 223, 234, 238])